## 🎲 Exact Winning Probabilities for the One- and Two-Dice Cases

This program computes the winning probabilities  
$$ P_A(V,W),\ P_B(V,W),\ P_U(V,W) $$  
for two given **betting strategies** $V$ and $W$, both **exactly** (using rational arithmetic) and **numerically rounded**, for the cases where

- one die is used,
- two dice are used.

---

### ✅ Requirements

- Both the **one-die** and **two-dice** cases use user-defined outcome probabilities that each sum to $1$.
- The strategies $V$ and $W$ specify how many chips are placed on the respective fields.

---

### ⚙️ Inputs

- `V, W`: the two betting strategies  
- `p`: probabilities of the individual fields  

---

### 🧩 Module `chip_strategies.py`

The file `chip_strategies.py` contains **all core functions** used for the computations:

- winning probabilities (`P_A`, `P_B`, `P_U`)
- strategy comparisons
- simulations and confidence intervals
- visualizations

> 📌 **Important:** This file must be located in the **root directory**  
> so that it can be imported in the notebooks  
> (e.g. `import chip_strategies as ps`).

The file `chip_strategies.py` can be regarded as a **function library** shared by all Jupyter notebooks in this project.

---

### ▶️ Next Steps

The corresponding Python code is located in the **next code cell**.  
Only the following inputs need to be adjusted:

- `V`, `W`: betting strategies of the two players  
  (e.g. `V = (3, 2, 1), W = (4, 2, 0)`)
- `p`: probabilities of the fields  
  (e.g. `p = [Fraction(1,2), Fraction(1,3), Fraction(1,6)]`)

> 📌 Make sure that `p`, `V`, and `W` have the same length.

---

### ▶️ Running the Program

- Click inside the code cell.
- Press `Shift + Enter` to start the computation.  
  Alternatively, click **▶ Run** in the toolbar at the top.
- Use **Run → Run All Cells** to execute all cells at once.

📎 Good luck experimenting with your own strategies!


In [2]:
from functools import lru_cache
from fractions import Fraction
import time

# ============================================================
# Input
# ============================================================

# Example strategies and probabilities
V = (3, 2, 1)
W = (4, 2, 0)

# Exact probabilities as Fractions
p = [Fraction(1,2), Fraction(1,3), Fraction(1,6)]

# ─────────────────────────────────────────────────────────────────────────────
# 1) One-die case: recursive exact winning probabilities
# ─────────────────────────────────────────────────────────────────────────────

def make_one_dice_recursion(p_list):
    """
    Constructs P_A(V,W) for the one-die case:
      P_A(V,W) = sum_{i in Act} (p_i / C) * P_A(V - e_i, W - e_i)
    with base cases and normalization C = sum_{i in Act} p_i.
    """
    @lru_cache(maxsize=None)
    def P_A1(V, W):
        V = list(V); W = list(W)
        # Base cases
        if sum(V) == 0 and sum(W) > 0:
            return Fraction(1)
        if sum(W) == 0:
            return Fraction(0)

        # Active fields
        Act = [i for i in range(len(p_list)) if V[i] + W[i] > 0]
        C = sum(p_list[i] for i in Act)

        x = Fraction(0)
        for i in Act:
            V2 = V.copy(); W2 = W.copy()
            if V2[i] > 0: V2[i] -= 1
            if W2[i] > 0: W2[i] -= 1
            x += Fraction(p_list[i], C) * P_A1(tuple(V2), tuple(W2))
        return x

    return P_A1

# Wrapper for (P_A, P_B, P_U)
def compute_one_dice(V, W, p_list):
    P_A1 = make_one_dice_recursion(p_list)
    PA = P_A1(tuple(V), tuple(W))
    PB = P_A1(tuple(W), tuple(V))
    PU = Fraction(1) - PA - PB
    return PA, PB, PU

# ─────────────────────────────────────────────────────────────────────────────
# 2) Two-dice case: recursive exact winning probabilities
# ─────────────────────────────────────────────────────────────────────────────

def make_two_dice_recursion(p1_list, p2_list):
    """
    Constructs P_A(V,W) for the two-dice case:
    sum over all pairs (i, j), remove chips if possible,
    and eliminate self-loops via p_stay.
    """
    @lru_cache(maxsize=None)
    def P_A2(V, W):
        V = list(V); W = list(W)
        # Base cases
        if sum(V) == 0 and sum(W) > 0:
            return Fraction(1)
        if sum(W) == 0:
            return Fraction(0)

        X = Fraction(0)
        p_stay = Fraction(0)
        m = len(p1_list)
        for i in range(m):
            for j in range(m):
                pij = p1_list[i] * p2_list[j]
                a_rem = V[i] > 0
                b_rem = W[j] > 0
                if not (a_rem or b_rem):
                    # Self-loop: no chip removed
                    p_stay += pij
                else:
                    V2 = V.copy(); W2 = W.copy()
                    if a_rem: V2[i] -= 1
                    if b_rem: W2[j] -= 1

                    # Absorbing cases after both moves
                    s2, t2 = sum(V2), sum(W2)
                    if s2 == 0 and t2 > 0:
                        X += pij  # A has now won
                    elif t2 == 0:
                        # B has won or a tie → P_A contribution = 0
                        pass
                    else:
                        X += pij * P_A2(tuple(V2), tuple(W2))

        # Eliminate self-loops
        return X / (1 - p_stay)

    return P_A2

# Wrapper for (P_A, P_B, P_U)
def compute_two_dice(V, W, p1_list, p2_list):
    P_A2 = make_two_dice_recursion(p1_list, p2_list)
    PA = P_A2(tuple(V), tuple(W))
    PB = P_A2(tuple(W), tuple(V))
    PU = Fraction(1) - PA - PB
    return PA, PB, PU

# ─────────────────────────────────────────────────────────────────────────────
# 3) Output
# ─────────────────────────────────────────────────────────────────────────────

print("\n=== ONE-DIE CASE (exact) ===")
t0 = time.perf_counter()
PA1, PB1, PU1 = compute_one_dice(V, W, p)
t1 = time.perf_counter()
print(f"Runtime: {t1 - t0:.3f}s")
print(f"P_A = {PA1} (≈ {float(PA1):.4f})")
print(f"P_B = {PB1} (≈ {float(PB1):.4f})")
print(f"P_U = {PU1} (≈ {float(PU1):.4f})")

print("\n=== TWO-DICE CASE (exact) ===")
t2 = time.perf_counter()
PA2, PB2, PU2 = compute_two_dice(V, W, p, p)
t3 = time.perf_counter()
print(f"Runtime: {t3 - t2:.3f}s")
print(f"P_A = {PA2} (≈ {float(PA2):.4f})")
print(f"P_B = {PB2} (≈ {float(PB2):.4f})")
print(f"P_U = {PU2} (≈ {float(PU2):.4f})\n")



=== ONE-DIE CASE (exact) ===
Runtime: 0.001s
P_A = 1181921/2400000 (≈ 0.4925)
P_B = 65/256 (≈ 0.2539)
P_U = 9511/37500 (≈ 0.2536)

=== TWO-DICE CASE (exact) ===
Runtime: 0.019s
P_A = 123236774559574168426903996056280400373103/281477364250855970833224885244827648000000 (≈ 0.4378)
P_B = 62849569545570383695296510944165062981331/140738682125427985416612442622413824000000 (≈ 0.4466)
P_U = 6508290120028207003145573460043424332847/56295472850171194166644977048965529600000 (≈ 0.1156)

